## Phase 3 Final Project Submission

Please fill out:
* Student name: Victor Kigen, 
* Student pace: Full time hybrid
* Scheduled project review date/time: 
* Instructor name: Mwikali
* Blog post URL:


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [3]:
# Loading Test set values data
test_set_values_data = pd.read_csv("test_set_values.csv")

# Loading Training set labels data
training_set_labels_data = pd.read_csv("training_set_labels.csv")

# Loading Training Set Values data
training_set_values_data = pd.read_csv("training_set_values.csv")